In [2]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import minimize

# Parameters
T = 5  # Total periods
Rf = 1.015
mu_R, sigma_R = 1.08, 0.18
beta = 0.95  # Discount factor
W0 = 1.0  # Initial wealth

# Wealth grid
W_min, W_max = 0.1, 10.0
N_W = 100
W_grid = np.logspace(np.log10(W_min), np.log10(W_max), N_W)

# Gauss-Hermite quadrature for returns
N_R = 5
z, w = np.polynomial.hermite.hermgauss(N_R)
R_nodes = mu_R + sigma_R * np.sqrt(2) * z
weights = w * np.exp(z**2) / np.sqrt(np.pi)

# Initialize value and policy functions
V = np.zeros((T + 1, N_W))
C_policy = np.zeros((T, N_W))
a_policy = np.zeros((T, N_W))

# Terminal condition: consume all wealth
V[T, :] = (W_grid**(-4)) / (-4)

# Backward recursion
for t in range(T-1, -1, -1):
    V_next_interp = interp1d(W_grid, V[t+1, :], bounds_error=False, fill_value='extrapolate')
    for i, W in enumerate(W_grid):
        def objective(x):
            κ, a = x
            κ = np.clip(κ, 0.01, 1.0)
            a = np.clip(a, 0.0, 1.0)
            C = κ * W
            S = W - C
            if S <= 0:
                return np.inf
            W_next = (a * R_nodes + (1 - a) * Rf) * S
            V_next_vals = V_next_interp(W_next)
            if np.any(np.isnan(V_next_vals)):
                return np.inf
            expected_V = np.dot(weights, V_next_vals)
            utility = (C**-4) / (-4)
            return - (utility + beta * expected_V)
        
        # Bounds and constraints
        bounds = [(0.01, 1.0), (0.0, 1.0)]
        res = minimize(objective, x0=np.array([0.5, 0.5]), bounds=bounds, method='SLSQP')
        
        if res.success:
            optimal_κ, optimal_a = res.x
            C_policy[t, i] = optimal_κ
            a_policy[t, i] = optimal_a
            # Update value function
            C = optimal_κ * W
            S = W - C
            W_next = (optimal_a * R_nodes + (1 - optimal_a) * Rf) * S
            V_next_vals = V_next_interp(W_next)
            expected_V = np.dot(weights, V_next_vals)
            V[t, i] = (C**-4)/(-4) + beta * expected_V
        else:
            V[t, i] = -np.inf
            C_policy[t, i] = 0.01
            a_policy[t, i] = 0.5

# Output optimal policies at t=0 (initial period)
print("Optimal Consumption Policy at t=1:", C_policy[0])
print("Optimal Portfolio Policy at t=1:", a_policy[0])

C:\Users\leocr\Projects\Economics\.venv\Lib\site-packages\scipy\optimize\_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\leocr\Projects\Economics\.venv\Lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
C:\Users\leocr\Projects\Economics\.venv\Lib\site-packages\scipy\optimize\_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Optimal Consumption Policy at t=1: [0.5        0.6355     0.5        0.5        0.5        0.67195
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.5        0.01       0.5
 0.5        0.5        0.5        0.01       0.01       0.01
 0.5        0.5        0.01       0.46194944 0.01       0.01
 0.5        0.5        0.01       0.01       0.01       0.01
 0.01       0.01       0.5        0.01       0.01       0.5
 0.01       0.01       0.5        0.5        0.5        0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.5        0.01       0.01       0.5        0.01       0.01
 0.5        0.5        0.2475895  0.5        0.22832329 0.5
 0.04854753 0.5        0.5        0.5        0.21424997 0.5
 0.5        0.5        0.5        0.13142474 0.5        0.5
 0.5        0.14843929 0.5        0.5        0.09708    0.09672534
 0.5        0.5        0.5        0.5        0.5        0.09666056
 0.49582018 0.44623957 0.09775134 0.09727